In [146]:
from langchain.document_loaders import WebBaseLoader

In [147]:
urls = ["https://goias.gov.br/social/perguntas-e-respostas-frequentes-aprendiz-do-futuro/",
        "https://goias.gov.br/social/perguntas-e-respostas-frequentes-dignidade/",
        "https://goias.gov.br/social/perguntas-e-respostas-frequentes-maes-de-goias/",
        "https://goias.gov.br/social/perguntas-e-respostas-frequentes-goias-por-elas/",
        "https://goias.gov.br/social/perguntas-e-respostas-frequentes-sistema-socioeducativo/",
        "https://goias.gov.br/social/perguntas-e-respostas-frequentes-auxilios-agua-e-energia-pao-e-leite/",
        "https://goias.gov.br/social/perguntas-e-respostas-frequentes-carteira-de-identificacao-do-autista2/",
        "https://goias.gov.br/social/bolsa-familia/",
        "https://goias.gov.br/social/cadastro-unico-cadunico/",
        "https://goias.gov.br/social/perguntas-e-respostas-frequentes-passaporte-intermunicipal-da-pessoa-idosa/",
        "https://goias.gov.br/social/perguntas-e-respostas-frequentes-passe-livre-intermunicipal-da-pessoa-com-deficiencia/",
        "https://goias.gov.br/social/perguntas-e-respostas-frequentes-centro-de-referencia-estadual-da-igualdade-crei/"]

inicio = """Categoria  

Perguntas e Respostas Frequentes"""

fim = """


 











 









 













 

 





 Governo na palma da mão"""

In [148]:
textos_brutos = []
textos_conteudo = []
textos_process = []

for url in urls:
    textos = WebBaseLoader(url)
    textos_brutos = textos.load()
    textos_conteudo.append(textos_brutos[0].page_content)

In [149]:
tamanho_inicio = len(inicio)
tamanho_final = len(fim)

for respostas_ind in textos_conteudo:
    for j in range(len(respostas_ind) - tamanho_inicio):
        if inicio in respostas_ind[j:j + tamanho_inicio]:
            começo_frase = j + tamanho_inicio
        if fim in respostas_ind[j:j+tamanho_final]:
            textos_process.append(respostas_ind[começo_frase + 6 : j])
            break
    else:
        print("Respostas n encontradas")

In [150]:
print(textos_process[0])

O que é o Programa Aprendiz do Futuro?
O Aprendiz do Futuro é uma iniciativa do Governo do Estado de Goiás, por meio da Secretaria de Estado de Desenvolvimento Social (SEDS), que tem como visão oferecer o maior programa de sócio-aprendizagem do Brasil e do mundo com foco em tecnologia, alta performance e impacto social na vida dos jovens em situação de vulnerabilidade.
Quem pode participar?
– Jovens de 14 até 15 anos e 11 meses, não sendo aplicada a regra, nesse caso, a aprendizes com deficiência. 
– Existe a prioridade para jovens em vulnerabilidade econômico-social, estudantes da rede pública ou bolsistas 100% de rede particular, jovens com renda familiar de até dois salários mínimos ou ½ Salário – per capita.
– O Programa prioriza, ainda, jovens membros de etnia tradicional, sendo indígena, quilombola, cigano ou afrodescendente, que sejam pertencentes a família que possua em sua composição mulher vítima de violência doméstica e sujeitos a medidas socioeducativos ou de proteção.
– O 